In [1]:
import simplesimdb  # A leightweight data creator and database manager
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml # for output files

pd.set_option('display.float_format', lambda x: '%.2e' % x)
# Type make solvers in the repository to have the executable available
database = simplesimdb.Manager( executable='./solvers', directory='data', filetype = 'yaml')

In [2]:
#define conversion function 
def orderToString(x): 
    if np.isnan(x) : return 'n/a'
    return'%.2f'% x

def errorToString(x):
    return '%.2e' % x

In [3]:
def create_solver() :
    return {
        "type" : "CG", # CG, LGMRES, BICGSTABl, Multigrid
        "eps" : 1e-6,
        "preconditioner" :{
            "type" : "diagonal" # None or diagonal
        },
        "max_iter" : 50000, # maximum number of iterations
        # for CG
        "check-every" : 1, # errror check every .. iterations
        # for LGMRES
        "inner_m" : 30,
        "outer_k" : 3,
        # for BICGSTABL
        "l_input" : 3
    }


In [4]:
def create_inputfile ():
    '''Create an input file for the solvers FELTOR code'''
    return {
        "grid" : {
            "n" : 3,
            "Nx" : 128, 
            "Ny" : 256,
        },
        "solver" : create_solver(),
        "equations":
        {
            "amp" : 0.9, # The closer to 1 the more difficult!   
        },
        "elliptic" :
        {
            "jfactor" : 1,
            "direction" : "forward" # forward or centered

        }    
    }


In [5]:
def make_dataframe():
    '''Make a pandas Dataframe of all simulations in the database
    
    Concatenates all inputs and outputs as serialized dicts
    '''
    inputs  = list()
    outputs =list()
    for data in database.table() :
        inputs.append( data)
        with open( database.outfile( data)) as f:
            output = yaml.full_load(f)
            outputs.append( output)
    dfi = pd.json_normalize(inputs)        
    dfo = pd.json_normalize(outputs)
    return pd.concat([dfi, dfo], axis=1).sort_values( 'grid.Nx', ascending=True)

In [6]:
# Clean out all existing simulations
database.delete_all()
database.directory='data'

In [7]:
string_dflist = []
string_directions = []
string_ns = []
ns = [3,4]
for n in ns :
    for direction in ['forward','backward','centered']:
        inputfile = create_inputfile()
        for (i,eps) in zip([17,34,68,136],[1e-6,1e-7,1e-8,1e-9]):
            inputfile["grid"]["n"] = n
            inputfile["grid"]["Nx"] = i
            inputfile["grid"]["Ny"] = i
            inputfile["solver"]["eps"] = eps
            inputfile["elliptic"]["direction"] = direction
            database.create( inputfile, error = 'display')

        df = make_dataframe()
        dff = df.loc [ (df["grid.n"]==n) & (df['elliptic.direction'] == direction)]
        dff = dff[['grid.n','grid.Nx','grid.Ny',
                  'iter','time','error','error_var_fw', 'error_var_bw', 'error_var_cc']]

        for name in ['error','error_var_fw', 'error_var_bw', 'error_var_cc']:
            dff.insert(dff.columns.get_loc(name)+1,column="order."+name, value=dff.loc[:,name])
            dff["order."+name] = (np.log ( dff["order."+name]/ dff["order."+name].shift(1))/
                np.log( dff["grid.Nx"].shift(1)/dff["grid.Nx"]))
        dff.set_index(['grid.n','grid.Nx','grid.Ny','iter','time'], inplace=True)
        headers = [ "$\phi$",
                    "$(\nabla\phi)^2_{FW}$",
                    "$(\nabla\phi)^2_{BW}$",
                    "$(\nabla\phi)^2_{CC}$"]
        dff.columns=pd.MultiIndex.from_product([headers,["error", "order"] ])
        #We would like the order to be displayed differently.
        #In pandas we must convert the columns to strings to do that.
        string_dff = dff.copy()
        for header in headers :
            string_dff[header, "error"]=string_dff[header, "error"].apply( errorToString)
            string_dff[header, "order"]=string_dff[header, "order"].apply( orderToString)
        string_dflist.append(string_dff)
        string_directions.append(direction)
        string_ns.append(n)

Running simulation ce55d3...edb8.yaml
Running simulation a85843...b013.yaml
Running simulation f2036e...e5da.yaml
Running simulation 0aef15...b3bf.yaml
Running simulation f8107f...d77c.yaml
Running simulation 1692e9...e489.yaml
Running simulation 05a3b7...7fe2.yaml
Running simulation 2aa3eb...76f0.yaml
Running simulation 3601a5...6e15.yaml
Running simulation a31d12...bb49.yaml
Running simulation 562091...c3d8.yaml
Running simulation b4ce5b...830e.yaml
Running simulation 72e3f7...1d78.yaml
Running simulation ec29ae...e4ab.yaml
Running simulation 91ae4f...9197.yaml
Running simulation 6482b5...30a4.yaml
Running simulation 2cd97b...7e2c.yaml
Running simulation d9be8b...625c.yaml
Running simulation 5e8c3f...2b9a.yaml
Running simulation dc7da6...e9d1.yaml
Running simulation 992e20...b529.yaml
Running simulation b737e0...6600.yaml
Running simulation 072554...ae84.yaml
Running simulation cab0b6...519f.yaml


In [8]:
for i in range(0,len(string_dflist)):
    print( f"Direction is {string_directions[i]}, n is {string_ns[i]}")
    display( string_dflist[i])

Direction is forward, n is 3


$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
3      17      17      241  3.43e-02  1.87e-04   n/a              9.65e-04   
       34      34      516  1.58e-01  2.32e-05  3.01              1.29e-04   
       68      68      1075 6.73e-01  2.92e-06  2.99              1.68e-05   
       136     136     2250 7.18e+00  3.67e-07  2.99              2.14e-06   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
3      17      17      241  3.43e-02   n/a              1.04e-02   n/a   
       34      34      516  1.58e-01  2.90              2.65e-03  1.97   
       68      68      1075 6.73e-01  2.94              6.71e-04  1.98   
       136     136     2250 7.18e+00  2.97              1.69e-04  1.99   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
3      17      17      241  3.43e-02              5.26e-03   n/a  
       34      34      516  1.58e-01              1.33e-03  1.98  
       68      68      1075 6.73e-01              3.36e-04  1.98  
       136     136     2250 7.18e+00              8.45e-05  1.99

Direction is backward, n is 3


$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
3      17      17      241  1.91e-02  1.87e-04   n/a              1.04e-02   
       34      34      514  1.00e-01  2.32e-05  3.01              2.65e-03   
       68      68      1074 7.93e-01  2.92e-06  2.99              6.71e-04   
       136     136     2243 1.07e+01  3.67e-07  2.99              1.69e-04   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
3      17      17      241  1.91e-02   n/a              9.65e-04   n/a   
       34      34      514  1.00e-01  1.97              1.29e-04  2.90   
       68      68      1074 7.93e-01  1.98              1.68e-05  2.94   
       136     136     2243 1.07e+01  1.99              2.14e-06  2.97   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
3      17      17      241  1.91e-02              5.26e-03   n/a  
       34      34      514  1.00e-01              1.33e-03  1.98  
       68      68      1074 7.93e-01              3.36e-04  1.98  
       136     136     2243 1.07e+01              8.45e-05  1.99

Direction is centered, n is 3


...

$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
4      17      17      395  4.53e-02  4.36e-06   n/a              4.40e-05   
       34      34      850  3.03e-01  2.60e-07  4.07              2.97e-06   
       68      68      1776 2.46e+00  1.62e-08  4.01              1.93e-07   
       136     136     3711 2.73e+01  1.02e-09  3.99              1.23e-08   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
4      17      17      395  4.53e-02   n/a              3.12e-04   n/a   
       34      34      850  3.03e-01  3.89              3.72e-05  3.07   
       68      68      1776 2.46e+00  3.94              4.61e-06  3.01   
       136     136     3711 2.73e+01  3.97              5.77e-07  3.00   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
4      17      17      395  4.53e-02              1.58e-04   n/a  
       34      34      850  3.03e-01              1.88e-05  3.07  
       68      68      1776 2.46e+00              2.32e-06  3.02  
       136     136     3711 2.73e+01              2.89e-07  3.00

Direction is backward, n is 4


$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
4      17      17      395  4.82e-02  4.36e-06   n/a              3.12e-04   
       34      34      848  3.43e-01  2.60e-07  4.07              3.72e-05   
       68      68      1772 2.49e+00  1.62e-08  4.01              4.61e-06   
       136     136     3713 2.72e+01  1.02e-09  3.99              5.77e-07   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
4      17      17      395  4.82e-02   n/a              4.40e-05   n/a   
       34      34      848  3.43e-01  3.07              2.97e-06  3.89   
       68      68      1772 2.49e+00  3.01              1.93e-07  3.94   
       136     136     3713 2.72e+01  3.00              1.23e-08  3.97   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
4      17      17      395  4.82e-02              1.58e-04   n/a  
       34      34      848  3.43e-01              1.88e-05  3.07  
       68      68      1772 2.49e+00              2.32e-06  3.02  
       136     136     3713 2.72e+01              2.89e-07  3.00

Direction is centered, n is 4


$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
4      17      17      258  3.48e-02  8.80e-06   n/a              5.03e-04   
       34      34      561  2.58e-01  6.71e-07  3.71              7.64e-05   
       68      68      1190 2.05e+00  4.70e-08  3.84              1.07e-05   
       136     136     2509 2.07e+01  3.12e-09  3.91              1.43e-06   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
4      17      17      258  3.48e-02   n/a              5.03e-04   n/a   
       34      34      561  2.58e-01  2.72              7.64e-05  2.72   
       68      68      1190 2.05e+00  2.84              1.07e-05  2.84   
       136     136     2509 2.07e+01  2.91              1.43e-06  2.91   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
4      17      17      258  3.48e-02              9.05e-05   n/a  
       34      34      561  2.58e-01              1.26e-05  2.84  
       68      68      1190 2.05e+00              1.72e-06  2.88  
       136     136     2509 2.07e+01              2.27e-07  2.92

We observe that the derivative, if computed with a different dG method, converges with one order less than the solution and the derivative with the correct dG method.
Further, we see that the centered derivative is order 3 for both n=3 and n=4 (!!)

In [9]:
string_dflist = []
string_directions = []
string_ns = []
ns = [3,4]
name_list = [['error_dxx_fw_fw', 'error_dxx_fw_bw', 'error_dxx_fw_cc',
                     'error_dxx_bw_fw', 'error_dxx_bw_bw', 'error_dxx_bw_cc',
                     'error_dxx_cc_fw', 'error_dxx_cc_bw', 'error_dxx_cc_cc'],
            [ 'error_dxy_fw_fw', 'error_dxy_fw_bw', 'error_dxy_fw_cc',
                     'error_dxy_bw_fw', 'error_dxy_bw_bw', 'error_dxy_bw_cc',
                     'error_dxy_cc_fw', 'error_dxy_cc_bw', 'error_dxy_cc_cc'],
             [ 'error_hess_fw_fw', 'error_hess_fw_bw', 'error_hess_fw_cc',
                    'error_hess_bw_fw', 'error_hess_bw_bw', 'error_hess_bw_cc',
                    'error_hess_cc_fw', 'error_hess_cc_bw', 'error_hess_cc_cc']
            ]
header_list = [["$\partial_x^2 \phi_{FW,FW}$", "$\partial_x^2 \phi_{FW,BW}$","$\partial_x^2 \phi_{FW,CC}$",
                "$\partial_x^2 \phi_{BW,FW}$", "$\partial_x^2 \phi_{BW,BW}$","$\partial_x^2 \phi_{BW,CC}$",
                "$\partial_x^2 \phi_{CC,FW}$", "$\partial_x^2 \phi_{CC,BW}$","$\partial_x^2 \phi_{CC,CC}$"],
["$\partial_y\partial_x\phi_{FW,FW}$","$\partial_y\partial_x\phi_{FW,BW}$","$\partial_y\partial_x\phi_{FW,CC}$",
"$\partial_y\partial_x\phi_{BW,FW}$","$\partial_y\partial_x\phi_{BW,BW}$","$\partial_y\partial_x\phi_{BW,CC}$",
"$\partial_y\partial_x\phi_{CC,FW}$","$\partial_y\partial_x\phi_{CC,BW}$","$\partial_y\partial_x\phi_{CC,CC}$"],
            ["$H \phi_{FW,FW}$", "$H \phi_{FW,BW}$","$H \phi_{FW,CC}$",
             "$H \phi_{BW,FW}$", "$H \phi_{BW,BW}$","$H \phi_{BW,CC}$",
             "$H \phi_{CC,FW}$", "$H \phi_{CC,BW}$","$H \phi_{CC,CC}$"]
            ]
for (names,headers) in zip(name_list, header_list):
    for n in ns :
        for direction in ['forward','centered']:
            inputfile = create_inputfile()
            for (i,eps) in zip([17,34,68,136],[1e-6,1e-7,1e-8,1e-9]):
                inputfile["grid"]["n"] = n
                inputfile["grid"]["Nx"] = i
                inputfile["grid"]["Ny"] = i
                inputfile["solver"]["eps"] = eps
                inputfile["elliptic"]["direction"] = direction
                database.create( inputfile, error = 'display')

            df = make_dataframe()
            dff = df.loc [ (df["grid.n"]==n) & (df['elliptic.direction'] == direction)]

            
            dff = dff[['grid.n','grid.Nx','grid.Ny'] + names]
            for name in names:
                dff.insert(dff.columns.get_loc(name)+1,column="order."+name, value=dff.loc[:,name])
                dff["order."+name] = (np.log ( dff["order."+name]/ dff["order."+name].shift(1))/
                    np.log( dff["grid.Nx"].shift(1)/dff["grid.Nx"]))
            dff.set_index(['grid.n','grid.Nx','grid.Ny'], inplace=True)
            dff.columns=pd.MultiIndex.from_product([headers,["error", "order"] ])
            #We would like the order to be displayed differently.
            #In pandas we must convert the columns to strings to do that.
            string_dff = dff.copy()
            for header in headers :
                string_dff[header, "error"]=string_dff[header, "error"].apply( errorToString)
                string_dff[header, "order"]=string_dff[header, "order"].apply( orderToString)
            string_dflist.append(string_dff)
            string_directions.append(direction)
            string_ns.append(n)

Existing simulation ce55d3...edb8.yaml
Existing simulation a85843...b013.yaml
Existing simulation f2036e...e5da.yaml
Existing simulation 0aef15...b3bf.yaml
Existing simulation 3601a5...6e15.yaml
Existing simulation a31d12...bb49.yaml
Existing simulation 562091...c3d8.yaml
Existing simulation b4ce5b...830e.yaml
Existing simulation 72e3f7...1d78.yaml
Existing simulation ec29ae...e4ab.yaml
Existing simulation 91ae4f...9197.yaml
Existing simulation 6482b5...30a4.yaml
Existing simulation 992e20...b529.yaml
Existing simulation b737e0...6600.yaml
Existing simulation 072554...ae84.yaml
Existing simulation cab0b6...519f.yaml
Existing simulation ce55d3...edb8.yaml
Existing simulation a85843...b013.yaml
Existing simulation f2036e...e5da.yaml
Existing simulation 0aef15...b3bf.yaml
Existing simulation 3601a5...6e15.yaml
Existing simulation a31d12...bb49.yaml
Existing simulation 562091...c3d8.yaml
Existing simulation b4ce5b...830e.yaml
Existing simulation 72e3f7...1d78.yaml
Existing simulation ec29a

In [10]:
pd.set_option('max_columns', None)
for i in range(0,len(string_dflist)):
    print( f"Direction is {string_directions[i]}, n is {string_ns[i]}")
    display( string_dflist[i])

Direction is forward, n is 3


$\partial_x^2 \phi_{FW,FW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         6.14e-03   n/a   
       34      34                         1.67e-03  1.88   
       68      68                         4.37e-04  1.93   
       136     136                        1.12e-04  1.96   

                       $\partial_x^2 \phi_{FW,BW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         6.37e-03   n/a   
       34      34                         1.75e-03  1.87   
       68      68                         4.60e-04  1.92   
       136     136                        1.18e-04  1.96   

                       $\partial_x^2 \phi_{FW,CC}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         6.06e-03   n/a   
       34      34                         1.68e-03  1.85   
       68      68                         4.44e-04  1.92   
       136     136                        1.14e-04  1.96   

                       $\partial_x^2 \phi_{BW,FW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         1.21e-01   n/a   
       34      34                         5.65e-02  1.10   
       68      68                         2.72e-02  1.05   
       136     136                        1.33e-02  1.03   

                       $\partial_x^2 \phi_{BW,BW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         1.23e-01   n/a   
       34      34                         5.72e-02  1.10   
       68      68                         2.74e-02  1.06   
       136     136                        1.34e-02  1.03   

                       $\partial_x^2 \phi_{BW,CC}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         8.49e-02   n/a   
       34      34                         4.18e-02  1.02   
       68      68                         2.08e-02  1.01   
       136     136                        1.04e-02  1.00   

                       $\partial_x^2 \phi_{CC,FW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         6.06e-02   n/a   
       34      34                         2.82e-02  1.10   
       68      68                         1.36e-02  1.06   
       136     136                        6.65e-03  1.03   

                       $\partial_x^2 \phi_{CC,BW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         6.23e-02   n/a   
       34      34                         2.88e-02  1.11   
       68      68                         1.38e-02  1.07   
       136     136                        6.71e-03  1.04   

                       $\partial_x^2 \phi_{CC,CC}$        
                                             error order  
grid.n grid.Nx grid.Ny                                    
3      17      17                         4.30e-02   n/a  
       34      34                         2.10e-02  1.04  
       68      68                         1.04e-02  1.01  
       136     136                        5.18e-03  1.01

Direction is centered, n is 3


$\partial_x^2 \phi_{FW,FW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         5.27e-02   n/a   
       34      34                         2.61e-02  1.01   
       68      68                         1.30e-02  1.00   
       136     136                        6.53e-03  1.00   

                       $\partial_x^2 \phi_{FW,BW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         4.97e-02   n/a   
       34      34                         2.54e-02  0.97   
       68      68                         1.29e-02  0.98   
       136     136                        6.48e-03  0.99   

                       $\partial_x^2 \phi_{FW,CC}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         3.81e-02   n/a   
       34      34                         1.98e-02  0.94   
       68      68                         1.01e-02  0.97   
       136     136                        5.11e-03  0.99   

                       $\partial_x^2 \phi_{BW,FW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         4.97e-02   n/a   
       34      34                         2.54e-02  0.97   
       68      68                         1.29e-02  0.98   
       136     136                        6.48e-03  0.99   

                       $\partial_x^2 \phi_{BW,BW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         5.27e-02   n/a   
       34      34                         2.61e-02  1.01   
       68      68                         1.30e-02  1.00   
       136     136                        6.53e-03  1.00   

                       $\partial_x^2 \phi_{BW,CC}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         3.81e-02   n/a   
       34      34                         1.98e-02  0.94   
       68      68                         1.01e-02  0.97   
       136     136                        5.11e-03  0.99   

                       $\partial_x^2 \phi_{CC,FW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         4.35e-03   n/a   
       34      34                         9.80e-04  2.15   
       68      68                         2.29e-04  2.10   
       136     136                        5.45e-05  2.07   

                       $\partial_x^2 \phi_{CC,BW}$        \
                                             error order   
grid.n grid.Nx grid.Ny                                     
3      17      17                         4.35e-03   n/a   
       34      34                         9.80e-04  2.15   
       68      68                         2.29e-04  2.10   
       136     136                        5.45e-05  2.07   

                       $\partial_x^2 \phi_{CC,CC}$        
                                             error order  
grid.n grid.Nx grid.Ny                                    
3      17      17                         3.48e-03   n/a  
       34      34                         8.06e-04  2.11  
       68      68                         1.93e-04  2.06  
       136     136                        4.72e-05  2.03

Direction is forward, n is 4


...

$H \phi_{FW,FW}$       $H \phi_{FW,BW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
3      17      17              5.26e-02   n/a         4.58e-02   n/a   
       34      34              2.53e-02  1.05         2.34e-02  0.97   
       68      68              1.26e-02  1.01         1.21e-02  0.95   
       136     136             6.30e-03  1.00         6.19e-03  0.97   

                       $H \phi_{FW,CC}$       $H \phi_{BW,FW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
3      17      17              3.90e-02   n/a         4.58e-02   n/a   
       34      34              1.93e-02  1.02         2.34e-02  0.97   
       68      68              9.77e-03  0.98         1.21e-02  0.95   
       136     136             4.94e-03  0.98         6.19e-03  0.97   

                       $H \phi_{BW,BW}$       $H \phi_{BW,CC}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
3      17      17              5.26e-02   n/a         3.90e-02   n/a   
       34      34              2.53e-02  1.05         1.93e-02  1.02   
       68      68              1.26e-02  1.01         9.77e-03  0.98   
       136     136             6.30e-03  1.00         4.94e-03  0.98   

                       $H \phi_{CC,FW}$       $H \phi_{CC,BW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
3      17      17              1.11e-02   n/a         1.11e-02   n/a   
       34      34              2.43e-03  2.19         2.43e-03  2.19   
       68      68              5.72e-04  2.09         5.72e-04  2.09   
       136     136             1.39e-04  2.04         1.39e-04  2.04   

                       $H \phi_{CC,CC}$        
                                  error order  
grid.n grid.Nx grid.Ny                         
3      17      17              1.01e-02   n/a  
       34      34              2.25e-03  2.16  
       68      68              5.23e-04  2.11  
       136     136             1.25e-04  2.06

Direction is forward, n is 4


$H \phi_{FW,FW}$       $H \phi_{FW,BW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
4      17      17              1.48e-03   n/a         1.78e-03   n/a   
       34      34              2.22e-04  2.74         2.48e-04  2.85   
       68      68              3.05e-05  2.86         3.28e-05  2.92   
       136     136             4.01e-06  2.93         4.22e-06  2.96   

                       $H \phi_{FW,CC}$       $H \phi_{BW,FW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
4      17      17              6.86e-04   n/a         1.51e-02   n/a   
       34      34              5.38e-05  3.67         3.90e-03  1.95   
       68      68              4.04e-06  3.73         9.94e-04  1.97   
       136     136             3.40e-07  3.57         2.51e-04  1.98   

                       $H \phi_{BW,BW}$       $H \phi_{BW,CC}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
4      17      17              1.45e-02   n/a         3.40e-03   n/a   
       34      34              3.86e-03  1.91         8.16e-04  2.06   
       68      68              9.92e-04  1.96         2.05e-04  1.99   
       136     136             2.51e-04  1.98         5.19e-05  1.98   

                       $H \phi_{CC,FW}$       $H \phi_{CC,BW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
4      17      17              7.63e-03   n/a         7.45e-03   n/a   
       34      34              1.97e-03  1.95         1.96e-03  1.92   
       68      68              5.01e-04  1.98         5.01e-04  1.97   
       136     136             1.26e-04  1.99         1.26e-04  1.99   

                       $H \phi_{CC,CC}$        
                                  error order  
grid.n grid.Nx grid.Ny                         
4      17      17              1.70e-03   n/a  
       34      34              4.07e-04  2.06  
       68      68              1.03e-04  1.99  
       136     136             2.60e-05  1.98

Direction is centered, n is 4


$H \phi_{FW,FW}$       $H \phi_{FW,BW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
4      17      17              1.04e-02   n/a         1.15e-02   n/a   
       34      34              3.14e-03  1.72         3.31e-03  1.80   
       68      68              8.69e-04  1.85         8.91e-04  1.89   
       136     136             2.29e-04  1.93         2.31e-04  1.94   

                       $H \phi_{FW,CC}$       $H \phi_{BW,FW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
4      17      17              3.29e-03   n/a         1.15e-02   n/a   
       34      34              7.31e-04  2.17         3.31e-03  1.80   
       68      68              1.78e-04  2.03         8.91e-04  1.89   
       136     136             4.51e-05  1.98         2.31e-04  1.94   

                       $H \phi_{BW,BW}$       $H \phi_{BW,CC}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
4      17      17              1.04e-02   n/a         3.29e-03   n/a   
       34      34              3.14e-03  1.72         7.31e-04  2.17   
       68      68              8.69e-04  1.85         1.78e-04  2.03   
       136     136             2.29e-04  1.93         4.51e-05  1.98   

                       $H \phi_{CC,FW}$       $H \phi_{CC,BW}$        \
                                  error order            error order   
grid.n grid.Nx grid.Ny                                                 
4      17      17              3.57e-03   n/a         3.57e-03   n/a   
       34      34              1.08e-03  1.72         1.08e-03  1.72   
       68      68              3.05e-04  1.83         3.05e-04  1.83   
       136     136             8.13e-05  1.91         8.13e-05  1.91   

                       $H \phi_{CC,CC}$        
                                  error order  
grid.n grid.Nx grid.Ny                         
4      17      17              4.36e-04   n/a  
       34      34              4.91e-05  3.15  
       68      68              6.44e-06  2.93  
       136     136             8.57e-07  2.91

In second derivatives it seems to be important that we apply the second derivative to a good first derivative.
It does not matter which direction the second derivative is, we always lose one order